In [21]:
import os, sys

sys.path.append('c:\\Projects\\taxi_predicts\\src\\')

In [22]:
%reload_ext autoreload
%autoreload 2

In [23]:
import config as config

In [1]:
import hopsworks
#now we connect to the feture_store and feature group to store the data

#connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

#connect to the feature store
feature_store = project.get_feature_store()

#connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)



c:\Users\dpetrik\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'config' is not defined

In [25]:
#feature view, we define how we want to fetch data from feature groups and merge it to get the final data set

try:
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all()
    )
except:
    print("Feature view already existed. Skip creation.")


feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME,
    version=config.FEATURE_VIEW_VERSION
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/549012/fs/544835/fv/time_series_hourly_feature_view/version/1


In [37]:
#we generate the training data with hopsowork method trainind_data

ts_data, _ = feature_view.training_data(
    description = "Time-series hourly taxi rides"
)

Finished: Reading data from Hopsworks, using Hive (1162.61s) 


In [38]:
ts_data.sort_values(by=['pickup_location_id', 'pickup_hour'], inplace=True)
ts_data

,pickup_hour,rides,pickup_location_id
1432567,2022-01-01 00:00:00,0,1
1139468,2022-01-01 01:00:00,0,1
92890,2022-01-01 02:00:00,0,1
4348646,2022-01-01 03:00:00,0,1
3404372,2022-01-01 04:00:00,1,1
...,...,...,...
159877,2024-03-05 09:00:00,0,265
124096,2024-03-05 10:00:00,0,265
184183,2024-03-05 11:00:00,0,265
124925,2024-03-05 12:00:00,0,265


In [39]:
from data import transform_ts_data_into_features_and_target

features, targets = transform_ts_data_into_features_and_target(
    ts_data,
    input_seq_len=24*28,
    step_size=23
)

features_and_target = features.copy()
features_and_target['target_rides_next_hour'] = targets

print(f'{features_and_target.shape=}')

100%|██████████| 265/265 [01:03<00:00,  4.20it/s]


198220
features_and_target.shape=(198220, 675)


In [40]:
#now we split it into training and test data

from datetime import date, timedelta
from pytz import timezone
import pandas as pd
from data_split import train_test_split

#we are taking the data from today until one month ago and split it
cutoff_date = pd.to_datetime(date.today() -timedelta(days=28*1))

print(f'{cutoff_date=}')

x_train, y_train, x_test, y_test = train_test_split(
    features_and_target,
    cutoff_date,
    target_column_name='target_rides_next_hour'
)

print(f'{x_train.shape=}')
print(f'{x_test.shape=}')
print(f'{y_train.shape=}')
print(f'{y_test.shape=}')

cutoff_date=Timestamp('2024-02-20 00:00:00')
x_train.shape=(194245, 674)
x_test.shape=(3975, 674)
y_train.shape=(194245,)
y_test.shape=(3975,)


In [41]:
#now we train the model with lgbmodel with feature engineering and hyperparameter tuning

import numpy as np
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import optuna #we use optuna for hyperparameters

from model import get_pipeline

def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on a TimeSeriesSplit
    """
    # pick hyper-parameters
    #we use optuna to find the best hyperparameters
    hyperparams = {
        "metric": 'mae',
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),   
    }
    
    #we create a traning validation splits
    tss = TimeSeriesSplit(n_splits=4)
    scores = []
    
    # sort X_train by `pikup_hour` inplace
    # so the TimeSeriesSplit will split the data in a consistent way
    x_train.sort_values('pickup_hour', inplace=True)

    for train_index, val_index in tss.split(x_train):

        # split data for training and validation
        x_train_, x_val = x_train.iloc[train_index, :], x_train.iloc[val_index,:]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]
        
        # train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(x_train_, y_train_)
        
        # evaluate the model
        y_pred = pipeline.predict(x_val)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)
   
    # Return the mean score
    return np.array(scores).mean()

In [42]:
#study

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=1)

[I 2024-03-19 11:17:39,898] A new study created in memory with name: no-name-82948379-11d7-4594-acf5-f38982eb61d5
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [45]:
best_params = study.best_trial.params
print(f'{best_params=}')

best_params={'num_leaves': 89, 'feature_fraction': 0.6693931619382161, 'bagging_fraction': 0.2097048168865949, 'min_child_samples': 67}


In [46]:
pipeline = get_pipeline(**best_params)
pipeline.fit(x_train, y_train)

[LightGBM] [Warning] feature_fraction is set=0.6693931619382161, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6693931619382161
[LightGBM] [Warning] bagging_fraction is set=0.2097048168865949, subsample=1.0 will be ignored. Current value: bagging_fraction=0.2097048168865949
[LightGBM] [Warning] feature_fraction is set=0.6693931619382161, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6693931619382161
[LightGBM] [Warning] bagging_fraction is set=0.2097048168865949, subsample=1.0 will be ignored. Current value: bagging_fraction=0.2097048168865949
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.611165 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 171901
[LightGBM] [Info] Number of data points in the train set: 194245, number of used features: 676
[LightGBM] [Info] Start training from sco

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x00000146BDD4FC70>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.2097048168865949,
                               feature_fraction=0.6693931619382161,
                               min_child_samples=67, num_leaves=89))])

In [47]:
predictions = pipeline.predict(x_test)
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

[LightGBM] [Warning] feature_fraction is set=0.6693931619382161, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6693931619382161
[LightGBM] [Warning] bagging_fraction is set=0.2097048168865949, subsample=1.0 will be ignored. Current value: bagging_fraction=0.2097048168865949
test_mae=27.2881


In [48]:
#now we need to push the model into hopsworks registry model ad not localy
%reload_ext autoreload
%autoreload 2
import joblib
from paths import MODELS_DIR

joblib.dump(pipeline, MODELS_DIR / 'model.pkl')

['C:\\Projects\\taxi_predicts\\models\\model.pkl']

In [49]:
#the modelSchema is method for hopsworks
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(x_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

In [58]:
#and now we push it to the model registry
#didnt work from video

model_registry = project.get_model_registry()

model = model_registry.sklearn.create_model(
    name='taxi_demand_predictor_next_hour',
    metrics={"test_mae": test_mae},
    description="LightGBM regressor with a bit of hyper-parameter tuning",
    input_example=x_train.sample(),
    model_schema=model_schema
)

model.save('../models/model.pkl')
# model.save(MODELS_DIR / 'model.pkl') #not working error 

Connected. Call `.close()` to terminate connection gracefully.


Uploading: 100.000%|██████████| 856996/856996 elapsed<00:03 remaining<00:00  2.76it/s]
Uploading: 100.000%|██████████| 4466/4466 elapsed<00:01 remaining<00:000:09,  2.33s/it]
Uploading: 100.000%|██████████| 60844/60844 elapsed<00:01 remaining<00:00
Model export complete: 100%|██████████| 6/6 [00:13<00:00,  2.23s/it]                   

Model created, explore it at https://c.app.hopsworks.ai:443/p/549012/models/taxi_demand_predictor_next_hour/1


Model(name: 'taxi_demand_predictor_next_hour', version: 1)